In [1]:
from ultralytics import YOLO
YOLO("yolov8n.pt")  # This will download the model


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [ ]:
import cv2
import time
import os
import threading
import numpy as np
from ultralytics import YOLO
from datetime import datetime
import playsound

from ultralytics import YOLO
YOLO("yolov8n.pt")  # This will download the model

# Load YOLOv8n model
model = YOLO("yolov8n.pt")

# Global variables for ROI selection
roi_points = []
drawing_done = False

# Create folder to save intruder frames
save_dir = "intruder_alerts"
os.makedirs(save_dir, exist_ok=True)

# To avoid multiple sound plays simultaneously
alarm_playing = False

def select_roi(event, x, y, flags, param):
    global roi_points, drawing_done
    if event == cv2.EVENT_LBUTTONDOWN and len(roi_points) < 4:
        roi_points.append((x, y))
        print(f"Point {len(roi_points)}: {x}, {y}")
        if len(roi_points) == 4:
            drawing_done = True

def play_alarm_sound():
    global alarm_playing
    try:
        playsound.playsound("mixkit-facility-alarm-sound-999.wav")
    finally:
        alarm_playing = False  # Reset flag after sound ends

# Initialize camera
cap = cv2.VideoCapture(1)  # Use 0 or 1 based on your webcam index
cv2.namedWindow("Select ROI")
cv2.setMouseCallback("Select ROI", select_roi)

# ROI Selection Phase
print("Click 4 points to define ROI...")
while not drawing_done:
    ret, frame = cap.read()
    if not ret:
        continue
    temp_frame = frame.copy()
    for point in roi_points:
        cv2.circle(temp_frame, point, 5, (0, 255, 0), -1)
    cv2.imshow("Select ROI", temp_frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyWindow("Select ROI")

# Create ROI mask
roi_contour = np.array(roi_points, dtype=np.int32)
roi_mask = np.zeros(frame.shape[:2], dtype=np.uint8)
cv2.fillPoly(roi_mask, [roi_contour], 255)

print("ROI selected. Starting detection...")

# Main detection loop at 1 FPS
while True:
    start_time = time.time()

    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, classes=[0], verbose=False)  # Only detect persons

    intruder_detected = False

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

            if roi_mask[cy, cx] == 255:
                intruder_detected = True
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
                cv2.putText(frame, "Intruder!", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # Draw ROI polygon
    cv2.polylines(frame, [roi_contour], isClosed=True, color=(0, 255, 0), thickness=2)

    if intruder_detected:
        print("🚨 Intruder Detected!")

        # Save frame
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = os.path.join(save_dir, f"intruder_{timestamp}.jpg")
        cv2.imwrite(filename, frame)

        # Start alarm if not already playing
        if not alarm_playing:
            alarm_playing = True
            threading.Thread(target=play_alarm_sound, daemon=True).start()

    cv2.imshow("AI Security Feed", frame)

    # Maintain ~1 FPS
    elapsed = time.time() - start_time
    delay = max(1.0 - elapsed, 0)
    if cv2.waitKey(int(delay * 1000)) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


Click 4 points to define ROI...


In [ ]:
import cv2
import time
import numpy as np
from ultralytics import YOLO
import playsound

# Load YOLOv8n model
model = YOLO("yolov8n.pt")  # Pretrained on COCO, class 0 = person

# Global variables for ROI selection
roi_points = []
drawing_done = False

# Mouse callback to get 4 points
def select_roi(event, x, y, flags, param):
    global roi_points, drawing_done
    if event == cv2.EVENT_LBUTTONDOWN and len(roi_points) < 4:
        roi_points.append((x, y))
        print(f"Point {len(roi_points)}: {x}, {y}")
        if len(roi_points) == 4:
            drawing_done = True

# Initialize camera
cap = cv2.VideoCapture(1)
cv2.namedWindow("Select ROI")
cv2.setMouseCallback("Select ROI", select_roi)

# Get first frame to select ROI
print("Click 4 points to define ROI...")
while not drawing_done:
    ret, frame = cap.read()
    if not ret:
        continue
    temp_frame = frame.copy()
    for point in roi_points:
        cv2.circle(temp_frame, point, 5, (0, 255, 0), -1)
    cv2.imshow("Select ROI", temp_frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyWindow("Select ROI")

# Create ROI mask
roi_contour = np.array(roi_points, dtype=np.int32)
roi_mask = np.zeros(frame.shape[:2], dtype=np.uint8)
cv2.fillPoly(roi_mask, [roi_contour], 255)

print("ROI selected. Starting detection...")

# Detection loop at 1 FPS
while True:
    start_time = time.time()

    ret, frame = cap.read()
    if not ret:
        break

    # Run detection
    results = model(frame, classes=[0], verbose=False)  # Only class 0 (person)

    intruder_detected = False

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])

            # Get center of the bounding box
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

            # Check if center is inside ROI
            if roi_mask[cy, cx] == 255:
                intruder_detected = True
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
                cv2.putText(frame, "Intruder!", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                            0.9, (0, 0, 255), 2)

    # Draw ROI
    cv2.polylines(frame, [roi_contour], isClosed=True, color=(0, 255, 0), thickness=2)

    if intruder_detected:
        print("🚨 Intruder Detected!")
        playsound.playsound("mixkit-facility-alarm-sound-999.wav")
        # wait for the sound to finish before continuing
        # time.sleep(2)

    cv2.imshow("AI Security Feed", frame)

    # Wait so that total loop time is approx 1 second
    elapsed = time.time() - start_time
    delay = max(1.0 - elapsed, 0)
    if cv2.waitKey(int(delay * 1000)) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


Click 4 points to define ROI...
Point 1: 348, 82
Point 2: 112, 73
Point 3: 89, 460
Point 4: 359, 465
ROI selected. Starting detection...
